# gelatin_box

In [39]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw, ImageFont
import io
import os


# Constants
MARKER_ID = 6
# Replace with your specific 3D object file path from google_16k in YCB models 
FILE_PATH = '/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/009_gelatin_box/google_16k/textured.obj'
# Save the Arimo zip file on your system and replace the FONT_PATH with yours 
FONT_PATH = "/home/jose/Downloads/Arimo/Arimo-VariableFont_wght.ttf"
# Change the output folder according to your wish.
OUTPUT_PDF_PATH = f'/home/jose/Downloads/{os.path.basename(os.path.dirname(os.path.dirname(FILE_PATH)))}.pdf'
FONT_SIZE = 30  # Adjust the size as needed
# New Constants for Font Sizes
TITLE_FONT_SIZE = 40
ANNOTATION_FONT_SIZE = 30
AXES_FONT_SIZE = 30
DISTANCE_CAPTION_FONT_SIZE = 40

PLOT_DPI = 410

# Parameterized values
ARUCO_SIZE = 500  # The size of the ARUCO marke
ARUCO_DICT = cv2.aruco.DICT_4X4_250  # The ARUCO dictionary

# Both for changing plot's position 
PLOT_X_OFFSET = -34  # The offset for the plot's x-position
PLOT_Y_OFFSET = -800  # The offset for the plot's y-position

# Both for changing Marker's position 
MARKER_X_OFFSET = 890  # The offset for the marker's x-position
MARKER_Y_POS = 0  # The y-position of the marker

# Both for changing Black lines's position 
INTERSECTION_X_OFFSET = 370  # The offset for the intersection's x-position
INTERSECTION_Y_OFFSET = 1300  # The offset for the intersection's y-position

X_RANGE = 0.210
Y_RANGE = 0.297
INTERVAL_VALUE = 20
X_AXIS_TEXT_COORDS = (1700, 3000)
Y_AXIS_TEXT_COORDS = (2100, 200)
# Constants for title position
TITLE_X_POS = 200
TITLE_Y_POS = 3150  # Or just calculate the exact position you want


# Additional Constants for annotation positions and increments
ANNOTATION_START_X = 200
ANNOTATION_START_Y = 3200
ANNOTATION_LINE_SPACING = 30

def add_annotations(draw, annotations, start_x, start_y, line_spacing, font_path, font_size):
    try:
        font = ImageFont.truetype(font_path, size=font_size)
    except OSError:
        font = ImageFont.load_default()

    y_pos = start_y
    for annotation in annotations:
        draw.text((start_x, y_pos), annotation, font=font, fill=(0, 0, 0))
        y_pos += line_spacing
        

def generate_aruco_marker(id, size=ARUCO_SIZE, dictionary=ARUCO_DICT):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[0][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    min_z = np.min(v[:100000, 2])  # Added this line for z-axis minimum value
    max_z = np.max(v[:100000, 2])  # Added this line for z-axis maximum value
    fig, ax = plt.subplots()
    
    
    ax.scatter(v[:100000,0],v[:100000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:100000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)

    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z
    
def draw_axes_on_a4(a4_img, v, width, height, X_RANGE, Y_RANGE, INTERVAL_VALUE, 
                    X_AXIS_TEXT_COORDS, Y_AXIS_TEXT_COORDS):
    draw = ImageDraw.Draw(a4_img)
    try:
        font = ImageFont.truetype(FONT_PATH, size=AXES_FONT_SIZE)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()  # If Arial isn't available, use the default PIL font

    draw.line([(0, height - 1), (width, height - 1)], fill=(0, 0, 0))  # X-axis
    draw.line([(0, 0), (0, height)], fill=(0, 0, 0))  # Y-axis
    
    x_min, x_max = min(v[:, 0]), max(v[:, 0])
    y_min, y_max = min(v[:, 1]), max(v[:, 1])

    x_ticks_interval = width // INTERVAL_VALUE
    y_ticks_interval = height // INTERVAL_VALUE

    x_value_interval = X_RANGE / INTERVAL_VALUE
    y_value_interval = Y_RANGE / INTERVAL_VALUE

    x_values_drawn = set()
    y_values_drawn = set()

    x_text_x, x_text_y = X_AXIS_TEXT_COORDS
    y_text_x, y_text_y = Y_AXIS_TEXT_COORDS
    
    draw.text((x_text_x, height - x_text_y), "X-Axis", font=font, fill=(0, 0, 0))
    draw.text((y_text_x, y_text_y), "Y-Axis", font=font, fill=(0, 0, 0))
    

    for i in range(0, width + 1, x_ticks_interval):
        value = x_min-0.07 + (i / width) * X_RANGE
        value_rounded = round(value, 2)
        if value_rounded not in x_values_drawn:
            x_values_drawn.add(value_rounded)
            draw.line([(i, height - 1), (i, height - 20)], fill=(0, 0, 0))
            draw.text((i - 10, height - 40), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    for i in range(0, height + 1, y_ticks_interval):
        value = y_max+0.15 - (i / height) * Y_RANGE
        value_rounded = round(value, 2)
        if value_rounded not in y_values_drawn:
            y_values_drawn.add(value_rounded)
            draw.line([(0, i), (20, i)], fill=(0, 0, 0))
            draw.text((25, i - 10), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    return a4_img

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = MARKER_ID 
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    marker_pil_img = marker_pil_img.rotate(-90, expand=1)
    file_path = FILE_PATH
    object_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))  

    fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z = plot_object(file_path)

    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=PLOT_DPI)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 + MARKER_X_OFFSET
    marker_y_pos = MARKER_Y_POS
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2 + PLOT_X_OFFSET
    plot_y_pos = a4_pixel_height - plot_pil_img.height + PLOT_Y_OFFSET 

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))

        # Adding the title to the A4 image
    draw = ImageDraw.Draw(a4_img)
    try:
        title_font = ImageFont.truetype(FONT_PATH, size=TITLE_FONT_SIZE)  # Adjust the size as needed
    except OSError:
        title_font = ImageFont.load_default()
    draw.text((TITLE_X_POS, TITLE_Y_POS), object_name, font=title_font, fill=(0, 0, 0))  # Adjust the position (50, 50) as needed

    
    
    # Draw the axes on the A4 image
    a4_img = draw_axes_on_a4(a4_img, v, a4_pixel_width, a4_pixel_height,X_RANGE, Y_RANGE, INTERVAL_VALUE, 
                    X_AXIS_TEXT_COORDS, Y_AXIS_TEXT_COORDS)

    # Adding annotations to the A4 sheet
    draw = ImageDraw.Draw(a4_img)
    try:
        annotation_font = ImageFont.truetype(FONT_PATH, size=ANNOTATION_FONT_SIZE)  # Adjust the size as needed
    except OSError:
        annotation_font = ImageFont.load_default()

    annotations = [
        f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}",
        f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}",
        f"Z Min: {min_z:.6f}",
        f"Z Max: {max_z:.6f}"
    ]

    add_annotations(draw, annotations, ANNOTATION_START_X, ANNOTATION_START_Y, 
                    ANNOTATION_LINE_SPACING, FONT_PATH, ANNOTATION_FONT_SIZE)

    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:100000,1]) - min(v[:100000,1]))
    intersection_pixel_y = (max(v[:100000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 + INTERSECTION_X_OFFSET
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + INTERSECTION_Y_OFFSET

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    try:
        font = ImageFont.truetype(FONT_PATH, size=DISTANCE_CAPTION_FONT_SIZE)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    draw.text((intersection_x_a4 + 10, (intersection_y_a4 + marker_y_pos + marker_pil_img.height) / 2), 
          "10 cm", font=font, fill=line_color)
    
    output_pdf_path = OUTPUT_PDF_PATH
    a4_img.save(output_pdf_path, "PDF", resolution=100.0)
    plt.close()

# foam_brick

In [1]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw, ImageFont
import io
import os


# Constants
MARKER_ID = 6
FILE_PATH = '/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/061_foam_brick/google_16k/textured.obj'
FONT_PATH = "/home/jose/Downloads/Arimo/Arimo-VariableFont_wght.ttf"
OUTPUT_PDF_PATH = f'/home/jose/Downloads/{os.path.basename(os.path.dirname(os.path.dirname(FILE_PATH)))}.pdf'
FONT_SIZE = 30  # Adjust the size as needed

# New Constants for Plot DPI and Font Sizes
PLOT_DPI = 275  # Updated DPI
TITLE_FONT_SIZE = 40
ANNOTATION_FONT_SIZE = 30
AXES_FONT_SIZE = 30
DISTANCE_CAPTION_FONT_SIZE = 40

# Position and Size Constants
PLOT_X_OFFSET = -34
PLOT_Y_OFFSET = -1350  # Updated Y Offset for plot
MARKER_X_OFFSET = 690  # Updated X Offset for marker
MARKER_Y_POS = 0
ARUCO_SIZE = 500  # The size of the ARUCO marker
ARUCO_DICT = cv2.aruco.DICT_4X4_250  # The ARUCO dictionary

# Intersection lines constants 
INTERSECTION_X_OFFSET = 280  # Updated
INTERSECTION_Y_OFFSET = 1300  # Updated

# Axes drawing constants
X_RANGE = 0.210
Y_RANGE = 0.297
INTERVAL_VALUE = 20
X_AXIS_TEXT_COORDS = (1700, 3000)  # Updated positions for axis labels
Y_AXIS_TEXT_COORDS = (2100, 200)   # Updated positions for axis labels

# Constants for title and annotations position
TITLE_X_POS = 200  # Updated X position for the title
TITLE_Y_POS = 3200-50  # Updated Y position for the title

# Annotations positioning constants
ANNOTATION_START_X = 200  # Updated X position for annotations
ANNOTATION_START_Y = 3200  # Updated starting Y position for annotations
ANNOTATION_LINE_SPACING = 30


def add_annotations(draw, annotations, start_x, start_y, line_spacing, font_path, font_size):
    try:
        font = ImageFont.truetype(font_path, size=font_size)
    except OSError:
        font = ImageFont.load_default()

    y_pos = start_y
    for annotation in annotations:
        draw.text((start_x, y_pos), annotation, font=font, fill=(0, 0, 0))
        y_pos += line_spacing
        

def generate_aruco_marker(id, size=ARUCO_SIZE, dictionary=ARUCO_DICT):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[0][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    min_z = np.min(v[:100000, 2])  # Added this line for z-axis minimum value
    max_z = np.max(v[:100000, 2])  # Added this line for z-axis maximum value
    fig, ax = plt.subplots()
    
    
    ax.scatter(v[:100000,0],v[:100000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:100000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)

    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z
    
def draw_axes_on_a4(a4_img, v, width, height, X_RANGE, Y_RANGE, INTERVAL_VALUE, 
                    X_AXIS_TEXT_COORDS, Y_AXIS_TEXT_COORDS):
    draw = ImageDraw.Draw(a4_img)
    try:
        font = ImageFont.truetype(FONT_PATH, size=AXES_FONT_SIZE)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()  # If Arial isn't available, use the default PIL font

    draw.line([(0, height - 1), (width, height - 1)], fill=(0, 0, 0))  # X-axis
    draw.line([(0, 0), (0, height)], fill=(0, 0, 0))  # Y-axis
    
    x_min, x_max = min(v[:, 0]), max(v[:, 0])
    y_min, y_max = min(v[:, 1]), max(v[:, 1])

    x_ticks_interval = width // INTERVAL_VALUE
    y_ticks_interval = height // INTERVAL_VALUE

    x_value_interval = X_RANGE / INTERVAL_VALUE
    y_value_interval = Y_RANGE / INTERVAL_VALUE

    x_values_drawn = set()
    y_values_drawn = set()

    x_text_x, x_text_y = X_AXIS_TEXT_COORDS
    y_text_x, y_text_y = Y_AXIS_TEXT_COORDS
    
    draw.text((x_text_x, height - x_text_y), "X-Axis", font=font, fill=(0, 0, 0))
    draw.text((y_text_x, y_text_y), "Y-Axis", font=font, fill=(0, 0, 0))
    

    for i in range(0, width + 1, x_ticks_interval):
        value = x_min-0.07 + (i / width) * X_RANGE
        value_rounded = round(value, 2)
        if value_rounded not in x_values_drawn:
            x_values_drawn.add(value_rounded)
            draw.line([(i, height - 1), (i, height - 20)], fill=(0, 0, 0))
            draw.text((i - 10, height - 40), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    for i in range(0, height + 1, y_ticks_interval):
        value = y_max+0.15 - (i / height) * Y_RANGE
        value_rounded = round(value, 2)
        if value_rounded not in y_values_drawn:
            y_values_drawn.add(value_rounded)
            draw.line([(0, i), (20, i)], fill=(0, 0, 0))
            draw.text((25, i - 10), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    return a4_img

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = MARKER_ID 
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    marker_pil_img = marker_pil_img.rotate(-90, expand=1)
    file_path = FILE_PATH
    object_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))  

    fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z = plot_object(file_path)

    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=PLOT_DPI)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 + MARKER_X_OFFSET
    marker_y_pos = MARKER_Y_POS
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2 + PLOT_X_OFFSET
    plot_y_pos = a4_pixel_height - plot_pil_img.height + PLOT_Y_OFFSET 

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))

        # Adding the title to the A4 image
    draw = ImageDraw.Draw(a4_img)
    try:
        title_font = ImageFont.truetype(FONT_PATH, size=TITLE_FONT_SIZE)  # Adjust the size as needed
    except OSError:
        title_font = ImageFont.load_default()
    draw.text((TITLE_X_POS, TITLE_Y_POS), object_name, font=title_font, fill=(0, 0, 0))  # Adjust the position (50, 50) as needed

    
    
    # Draw the axes on the A4 image
    a4_img = draw_axes_on_a4(a4_img, v, a4_pixel_width, a4_pixel_height,X_RANGE, Y_RANGE, INTERVAL_VALUE, 
                    X_AXIS_TEXT_COORDS, Y_AXIS_TEXT_COORDS)

    # Adding annotations to the A4 sheet
    draw = ImageDraw.Draw(a4_img)
    try:
        annotation_font = ImageFont.truetype(FONT_PATH, size=ANNOTATION_FONT_SIZE)  # Adjust the size as needed
    except OSError:
        annotation_font = ImageFont.load_default()

    annotations = [
        f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}",
        f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}",
        f"Z Min: {min_z:.6f}",
        f"Z Max: {max_z:.6f}"
    ]

    add_annotations(draw, annotations, ANNOTATION_START_X, ANNOTATION_START_Y, 
                    ANNOTATION_LINE_SPACING, FONT_PATH, ANNOTATION_FONT_SIZE)

    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:100000,1]) - min(v[:100000,1]))
    intersection_pixel_y = (max(v[:100000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 + INTERSECTION_X_OFFSET
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + INTERSECTION_Y_OFFSET

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    try:
        font = ImageFont.truetype(FONT_PATH, size=DISTANCE_CAPTION_FONT_SIZE)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    draw.text((intersection_x_a4 + 10, (intersection_y_a4 + marker_y_pos + marker_pil_img.height) / 2), 
          "10 cm", font=font, fill=line_color)
    
    output_pdf_path = OUTPUT_PDF_PATH
    a4_img.save(output_pdf_path, "PDF", resolution=100.0)
    plt.close()

# mustard_bottle

In [23]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw, ImageFont
import io
import os


# Constants
MARKER_ID = 6
# Replace with your specific 3D object file path from google_16k in YCB models 
FILE_PATH = '/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/006_mustard_bottle/google_16k/textured.obj'
# Save the Arimo zip file on your system and replace the FONT_PATH with yours 
FONT_PATH = "/home/jose/Downloads/Arimo/Arimo-VariableFont_wght.ttf"
# Change the output folder according to your wish.
OUTPUT_PDF_PATH = f'/home/jose/Downloads/{os.path.basename(os.path.dirname(os.path.dirname(FILE_PATH)))}.pdf'
FONT_SIZE = 30  # Adjust the size as needed
PLOT_DPI = 275

# Parameterized values
ARUCO_SIZE = 500
ARUCO_DICT = cv2.aruco.DICT_4X4_250
PLOT_X_OFFSET = -34
PLOT_Y_OFFSET = -900
MARKER_X_OFFSET = 780
MARKER_Y_POS = 0
INTERSECTION_X_OFFSET = 290
INTERSECTION_Y_OFFSET = 1300
# ANNOTATIONS_Y_OFFSET = 150


def generate_aruco_marker(id, size=ARUCO_SIZE, dictionary=ARUCO_DICT):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[0][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    min_z = np.min(v[:100000, 2])  # Added this line for z-axis minimum value
    max_z = np.max(v[:100000, 2])  # Added this line for z-axis maximum value
    fig, ax = plt.subplots()
    
    
    ax.scatter(v[:100000,0],v[:100000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:100000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)

    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z
    
def draw_axes_on_a4(a4_img, v, width, height):
    draw = ImageDraw.Draw(a4_img)
    try:
        font = ImageFont.truetype(FONT_PATH, size=30)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()  # If Arial isn't available, use the default PIL font

    draw.line([(0, height - 1), (width, height - 1)], fill=(0, 0, 0))  # X-axis
    draw.line([(0, 0), (0, height)], fill=(0, 0, 0))  # Y-axis
    
    x_min, x_max = min(v[:, 0]), max(v[:, 0])
    y_min, y_max = min(v[:, 1]), max(v[:, 1])

    x_ticks_interval = width // 20
    y_ticks_interval = height // 20

    x_range = 0.210
    y_range = 0.297

    x_value_interval = x_range / 20
    y_value_interval = y_range / 20

    x_values_drawn = set()
    y_values_drawn = set()
    
    draw.text((1700, height - 3000), "X-Axis", font=font, fill=(0, 0, 0))
    draw.text((2100, 200), "Y-Axis", font=font, fill=(0, 0, 0))
    

    for i in range(0, width + 1, x_ticks_interval):
        value = x_min-0.07 + (i / width) * x_range
        value_rounded = round(value, 2)
        if value_rounded not in x_values_drawn:
            x_values_drawn.add(value_rounded)
            draw.line([(i, height - 1), (i, height - 20)], fill=(0, 0, 0))
            draw.text((i - 10, height - 40), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    for i in range(0, height + 1, y_ticks_interval):
        value = y_max+0.15 - (i / height) * y_range
        value_rounded = round(value, 2)
        if value_rounded not in y_values_drawn:
            y_values_drawn.add(value_rounded)
            draw.line([(0, i), (20, i)], fill=(0, 0, 0))
            draw.text((25, i - 10), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    return a4_img

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = MARKER_ID 
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    marker_pil_img = marker_pil_img.rotate(-90, expand=1)
    file_path = FILE_PATH
    object_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))  

    fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z = plot_object(file_path)
    
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=PLOT_DPI)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 + MARKER_X_OFFSET
    marker_y_pos = MARKER_Y_POS
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2 + PLOT_X_OFFSET
    plot_y_pos = a4_pixel_height - plot_pil_img.height + PLOT_Y_OFFSET

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))

    # Adding the title to the A4 image
    draw = ImageDraw.Draw(a4_img)
    try:
        title_font = ImageFont.truetype(FONT_PATH, size=40)  # Adjust the size as needed
    except OSError:
        title_font = ImageFont.load_default()
    draw.text((200,3200-50), object_name, font=title_font, fill=(0, 0, 0))  # Adjust the position (50, 50) as needed
    
    
    # Draw the axes on the A4 image
    a4_img = draw_axes_on_a4(a4_img, v, a4_pixel_width, a4_pixel_height)

    # Adding annotations to the A4 sheet
    draw = ImageDraw.Draw(a4_img)
    try:
        annotation_font = ImageFont.truetype(FONT_PATH, size=30)  # Adjust the size as needed
    except OSError:
        annotation_font = ImageFont.load_default()

    annotations_y_position = a4_pixel_height - ANNOTATIONS_Y_OFFSET  # Adjust this value to move annotations up or down
    x_axis_annotation = f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}"
    y_axis_annotation = f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}"
    z_min_annotation = f"Z Min: {min_z:.6f}"
    z_max_annotation = f"Z Max: {max_z:.6f}"

    draw.text((200, 3200), x_axis_annotation, font=annotation_font, fill=(0, 0, 0))
    draw.text((200, 3200 + 30), y_axis_annotation, font=annotation_font, fill=(0, 0, 0))
    draw.text((200, 3200 + 60), z_min_annotation, font=annotation_font, fill=(0, 0, 0))
    draw.text((200, 3200 + 90), z_max_annotation, font=annotation_font, fill=(0, 0, 0))

    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:100000,1]) - min(v[:100000,1]))
    intersection_pixel_y = (max(v[:100000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 + INTERSECTION_X_OFFSET
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + INTERSECTION_Y_OFFSET
    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    try:
        font = ImageFont.truetype(FONT_PATH, size=40)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    draw.text((intersection_x_a4 + 10, (intersection_y_a4 + marker_y_pos + marker_pil_img.height) / 2), 
          "10 cm", font=font, fill=line_color)
    
    output_pdf_path = OUTPUT_PDF_PATH
    a4_img.save(output_pdf_path, "PDF", resolution=100.0)
    plt.close()

In [24]:
import cv2
import pywavefront
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull
from PIL import Image, ImageDraw, ImageFont
import io
import os


# Constants
MARKER_ID = 6
# Replace with your specific 3D object file path from google_16k in YCB models 
FILE_PATH = '/home/jose/Downloads/GraspBenchmarkWorkspace/YCB/models/ycb/037_scissors/google_16k/textured.obj'
# Save the Arimo zip file on your system and replace the FONT_PATH with yours 
FONT_PATH = "/home/jose/Downloads/Arimo/Arimo-VariableFont_wght.ttf"
# Change the output folder according to your wish.
OUTPUT_PDF_PATH = f'/home/jose/Downloads/{os.path.basename(os.path.dirname(os.path.dirname(FILE_PATH)))}.pdf'
FONT_SIZE = 30  # Adjust the size as needed
PLOT_DPI = 780

# Parameterized values
ARUCO_SIZE = 500
ARUCO_DICT = cv2.aruco.DICT_4X4_250
PLOT_X_OFFSET = -34
PLOT_Y_OFFSET = 400
MARKER_X_OFFSET = 800
MARKER_Y_POS = 0
INTERSECTION_X_OFFSET = 190
INTERSECTION_Y_OFFSET = 1300
# ANNOTATIONS_Y_OFFSET = 150


def generate_aruco_marker(id, size=500, dictionary=cv2.aruco.DICT_4X4_250):
    aruco_dict = cv2.aruco.Dictionary_get(dictionary)
    marker_img = cv2.aruco.drawMarker(aruco_dict, id, size)
    return marker_img

def plot_object(file_path):
    scene = pywavefront.Wavefront(file_path)
    items = list(scene.materials.items())
    ver = items[0][1].vertices
    n_ver = int(len(ver)/8)
    v = np.zeros((n_ver,3))
    for i in range(n_ver):
        v[i,:] = ver[8*i+5:8*i+5+3]
    v = v[np.argsort(v[:,2]),:]
    min_z = np.min(v[:100000, 2])  # Added this line for z-axis minimum value
    max_z = np.max(v[:100000, 2])  # Added this line for z-axis maximum value
    fig, ax = plt.subplots()
    
    
    ax.scatter(v[:100000,0],v[:100000,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    ax.plot([0,0.01],[0,0],c='r')
    ax.plot([0,0],[0,0.01],c='g')
    ax.set_aspect('equal', 'box')

    hull = ConvexHull(v[:100000, :2])
    for simplex in hull.simplices:
        plt.plot(hull.points[simplex, 0], hull.points[simplex, 1], 'k-')
    polygon = Polygon(hull.points[hull.vertices], fill=None, edgecolor='r', linewidth=1)
    ax.add_patch(polygon)
    
    points = hull.points[hull.vertices]
    x1, y1 = points[0]
    x2, y2 = points[1]
    m = (y2 - y1) / (x2 - x1)
    angle_with_x_axis = np.degrees(np.arctan(m))
    angle_with_y_axis = 90 - abs(angle_with_x_axis)

    x_ticks = np.arange(min(v[:,0]), max(v[:,0])+0.02, 0.02)
    y_ticks = np.arange(min(v[:,1]), max(v[:,1])+0.02, 0.02)
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    return fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z
    
def draw_axes_on_a4(a4_img, v, width, height):
    draw = ImageDraw.Draw(a4_img)
    try:
        font = ImageFont.truetype(FONT_PATH, size=30)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()  # If Arial isn't available, use the default PIL font

    draw.line([(0, height - 1), (width, height - 1)], fill=(0, 0, 0))  # X-axis
    draw.line([(0, 0), (0, height)], fill=(0, 0, 0))  # Y-axis
    
    x_min, x_max = min(v[:, 0]), max(v[:, 0])
    y_min, y_max = min(v[:, 1]), max(v[:, 1])

    x_ticks_interval = width // 20
    y_ticks_interval = height // 20

    x_range = 0.210
    y_range = 0.297

    x_value_interval = x_range / 20
    y_value_interval = y_range / 20

    x_values_drawn = set()
    y_values_drawn = set()
    
    draw.text((1700, height - 3000), "X-Axis", font=font, fill=(0, 0, 0))
    draw.text((2100, 200), "Y-Axis", font=font, fill=(0, 0, 0))
    

    for i in range(0, width + 1, x_ticks_interval):
        value = x_min-0.07 + (i / width) * x_range
        value_rounded = round(value, 2)
        if value_rounded not in x_values_drawn:
            x_values_drawn.add(value_rounded)
            draw.line([(i, height - 1), (i, height - 20)], fill=(0, 0, 0))
            draw.text((i - 10, height - 40), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    for i in range(0, height + 1, y_ticks_interval):
        value = y_max+0.15 - (i / height) * y_range
        value_rounded = round(value, 2)
        if value_rounded not in y_values_drawn:
            y_values_drawn.add(value_rounded)
            draw.line([(0, i), (20, i)], fill=(0, 0, 0))
            draw.text((25, i - 10), f"{value_rounded:.2f}", font=font, fill=(0, 0, 0))

    return a4_img

def get_intersection_point():
    return 0, 0

if __name__ == "__main__":
    marker_id = MARKER_ID 
    marker_img = generate_aruco_marker(marker_id)
    marker_pil_img = Image.fromarray(marker_img)
    marker_pil_img = marker_pil_img.rotate(-90, expand=1)
    file_path = FILE_PATH
    object_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))  

    fig, v,angle_with_x_axis, angle_with_y_axis, min_z, max_z = plot_object(file_path)
    
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=PLOT_DPI)
    buf.seek(0)
    plot_pil_img = Image.open(buf)
    
    a4_pixel_width = int(8.27 * 300)
    a4_pixel_height = int(11.69 * 300)
    a4_img = Image.new('RGB', (a4_pixel_width, a4_pixel_height), (255, 255, 255))

    marker_x_pos = (a4_pixel_width - marker_pil_img.width) // 2 - plot_pil_img.width // 6 + MARKER_X_OFFSET
    marker_y_pos = MARKER_Y_POS
    plot_x_pos = (a4_pixel_width - plot_pil_img.width) // 2 + PLOT_X_OFFSET
    plot_y_pos = a4_pixel_height - plot_pil_img.height + PLOT_Y_OFFSET

    a4_img.paste(marker_pil_img, (marker_x_pos, marker_y_pos))
    a4_img.paste(plot_pil_img, (plot_x_pos, plot_y_pos))

    # Adding the title to the A4 image
    draw = ImageDraw.Draw(a4_img)
    try:
        title_font = ImageFont.truetype(FONT_PATH, size=40)  # Adjust the size as needed
    except OSError:
        title_font = ImageFont.load_default()
    draw.text((200,3200-50), object_name, font=title_font, fill=(0, 0, 0))  # Adjust the position (50, 50) as needed
    
    
    # Draw the axes on the A4 image
    a4_img = draw_axes_on_a4(a4_img, v, a4_pixel_width, a4_pixel_height)

    # Adding annotations to the A4 sheet
    draw = ImageDraw.Draw(a4_img)
    try:
        annotation_font = ImageFont.truetype(FONT_PATH, size=30)  # Adjust the size as needed
    except OSError:
        annotation_font = ImageFont.load_default()

    # annotations_y_position = a4_pixel_height - ANNOTATIONS_Y_OFFSET # Adjust this value to move annotations up or down
    x_axis_annotation = f"Angle with x-axis (degrees): {angle_with_x_axis:.2f}"
    y_axis_annotation = f"Angle with y-axis (degrees): {angle_with_y_axis:.2f}"
    z_min_annotation = f"Z Min: {min_z:.6f}"
    z_max_annotation = f"Z Max: {max_z:.6f}"

    draw.text((200, 3200), x_axis_annotation, font=annotation_font, fill=(0, 0, 0))
    draw.text((200, 3200 + 30), y_axis_annotation, font=annotation_font, fill=(0, 0, 0))
    draw.text((200, 3200 + 60), z_min_annotation, font=annotation_font, fill=(0, 0, 0))
    draw.text((200, 3200 + 90), z_max_annotation, font=annotation_font, fill=(0, 0, 0))

    intersection_x, intersection_y = get_intersection_point()
    y_scale_factor = plot_pil_img.height / (max(v[:100000,1]) - min(v[:100000,1]))
    intersection_pixel_y = (max(v[:100000, 1]) - intersection_y) * y_scale_factor

    intersection_x_a4 = plot_x_pos + plot_pil_img.width / 2 + INTERSECTION_X_OFFSET 
    intersection_y_a4 = plot_y_pos + intersection_pixel_y + INTERSECTION_Y_OFFSET 

    draw = ImageDraw.Draw(a4_img)
    line_color = (0, 0, 0)
    try:
        font = ImageFont.truetype(FONT_PATH, size=40)  # Try to use Arial font
    except OSError:
        font = ImageFont.load_default()
    draw.line([(intersection_x_a4, intersection_y_a4), 
               (intersection_x_a4, marker_y_pos + marker_pil_img.height)], fill=line_color)
    draw.text((intersection_x_a4 + 10, (intersection_y_a4 + marker_y_pos + marker_pil_img.height) / 2), 
          "10 cm", font=font, fill=line_color)
    
    output_pdf_path = OUTPUT_PDF_PATH
    a4_img.save(output_pdf_path, "PDF", resolution=100.0)
    plt.close()